# **1 数据集预处理** 

根据观察，该数据集除路径（URI）和参数外其他 Header 无任何攻击 Payload，具有很多冗余信息。因此对该数据集进行格式化，只保留 HTTP 方法、路径和参数。

In [1]:
import urllib.parse

normal_file_raw = '/Users/yalinghu/Desktop/tfm_2021/data/normalTrafficTraining.txt'
anomalous_file_raw = '/Users/yalinghu/Desktop/tfm_2021/data/anomalousTrafficTest.txt'

normal_file_pre = 'normal.txt'
anomalous_file_pre = 'anomalous.txt'


def pre_file(file_in, file_out=None):
    with open(file_in, 'r', encoding='utf-8') as f_in:
        lines = f_in.readlines()
    res = []
    for i in range(len(lines)):
        line = lines[i].strip()
        # 提取 GET类型的数据
        if line.startswith("GET"):
            res.append("GET " + line.split(" ")[1])
        # 提取 POST类型的数据
        elif line.startswith("POST") or line.startswith("PUT"):
            method = line.split(' ')[0]
            url = line.split(' ')[1]
            j = 1
            # 提取 POST包中的数据
            while True:
                # 定位消息正文的位置
                if lines[i + j].startswith("Content-Length"):
                    break
                j += 1
            j += 2
            data = lines[i + j].strip()
            url += '?'+data
            res.append(method + ' ' + url)

    with open(file_out, 'w', encoding='utf-8') as f_out:
        for line in res:
            line = urllib.parse.unquote(line, encoding='ascii', errors='ignore').replace('\n', '').lower()
            f_out.writelines(line + '\n')

    print("{}数据预提取完成 {}条记录".format(file_out, len(res)))


if __name__ == '__main__':
    pre_file(normal_file_raw, normal_file_pre)
    pre_file(anomalous_file_raw, anomalous_file_pre)

normal.txt数据预提取完成 36000条记录
anomalous.txt数据预提取完成 25065条记录


# **2 特征提取**
经过上面的处理我们有了格式化的文本数据，但是如果想要运用机器学习的方法，单单做到这里是不够的，我们还需要进一步地对这里的文本数据进行特征提取。一个 Web 访问记录的成分是比较固定的，每个部分（方法、路径、参数、HTTP 头、Cookie 等）都有比较好的结构化特点，因此可以把 Web 攻击识别任务抽象为文本分类任务。这里我们采用两钟方法进行特征提取。


## 2.1 TF-IDF
对文本数据进行分类，常用的方法有词袋模型、TF-IDF、词向量化等方法。由于只是一次简单的知识梳理实验，这里就不对其进行比较验证了，直接选择TF-IDF对文本数据进行向量化处理。

在进行向量化处理之前，先对文本数据进行整合、打标签的操作


In [4]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = f.readlines()
    result = []
    for d in data:
        d = d.strip()
        if len(d) > 0:
            result.append(d)
    return result

normal_requests = load_data('normal.txt')
anomalous_requests = load_data('anomalous.txt')

all_requests = normal_requests + anomalous_requests
y_normal = [0] * len(normal_requests)
y_anomalous = [1] * len(anomalous_requests)
y = y_normal + y_anomalous

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="word", sublinear_tf=True)
X = vectorizer.fit_transform(all_requests)

In [6]:
vectorizer.vocabulary_

{'get': 19606,
 'http': 20938,
 'localhost': 23232,
 '8080': 8750,
 'tienda1': 31274,
 'index': 21282,
 'jsp': 21914,
 'publico': 27735,
 'anadir': 11513,
 'id': 21106,
 'nombre': 25744,
 'vino': 32658,
 'rioja': 28665,
 'precio': 27578,
 '100': 1079,
 'cantidad': 14443,
 '55': 6608,
 'b1': 12410,
 'aadir': 10396,
 'al': 10904,
 'carrito': 14650,
 'post': 27513,
 'autenticar': 12288,
 'modo': 24778,
 'entrar': 17935,
 'login': 23252,
 'choong': 15302,
 'pwd': 27837,
 'd1se3cin': 16347,
 'remember': 28400,
 'off': 25987,
 'caracteristicas': 14510,
 'errormsg': 18064,
 'credenciales': 16098,
 'incorrectas': 21263,
 'miembros': 24577,
 'pagar': 26485,
 'insertar': 21368,
 '2672': 3274,
 'pasar': 26717,
 'por': 27465,
 'caja': 14192,
 'productos': 27660,
 'registro': 28345,
 'cen': 14990,
 'password': 26740,
 '40a5e': 4953,
 'nurit': 25864,
 'apellidos': 11723,
 'ferrandez': 18694,
 'cabaas': 14099,
 'email': 17719,
 'tarpey': 30952,
 'bwds': 13986,
 'ne': 25477,
 'dni': 17096,
 '17621392b

In [8]:
X.shape

(61065, 33550)

可以看到特征维度为33550，这样一个维度对于很多机器算法来说计算复杂度太高，也很容易产生过拟合的情况。当然遇到这种情况，应该采用相应的手段对其进行降维操作，但是由于机器限制在保留90%信息的情况下得出所需的维度需要相当长的时间，故直接采用这一维度进行实验。



## 2.2 自定义特征向量

在上面用TF-IDF进行特征提取时，遇到的问题是数据的维度太高，虽然我们还没有用机器学习算法进行实际的训练但是这样高的维度让我们应该考虑一些备用方案，比如尝试进行提取自定义的特征。

这里我们不妨对最初的文本数据进行如下的特征提取：

URL长度  
请求类型  
参数部分长度  
参数的个数  
参数的最大长度  
参数的数字个数  
参数值中数字所占比例  
参数值中字母所占比例  
特殊字符个数  
特殊字符所占比例  




In [44]:
def vector(data):
    feature = []

    for i in range(len(data)):
        # 采用 split("?", 1)是为了处理形如 http://127.0.0.1/?open=?123的URL
        s = data[i].split("?", 1)
        if len(s) != 1:

            url_len = len(data[i])

            method = data[i].split(" ")[0]
            if method == "get":
                url_method = 1
            elif method == "post":
                url_method = 2
            else:
                url_method = 3

            query = s[1]
            parameter_len = len(query)
            parameters = query.split("&")
            parameter_num = len(parameters)

            parameter_max_len = 0
            parameter_number_num = 0
            parameter_str_num = 0
            parameter_spe_num = 0
            par_val_sum = 0

            for parameter in parameters:
                try:
                    # 采用 split("=", 1)是为了处理形如 open=123=234&file=op的参数
                    [par_name, par_val] = parameter.split("=", 1)
                except ValueError as err:
                    # 处理形如 ?open 这样的参数
                    print(err)
                    print(data[i])
                    break

                par_val_sum += len(par_val)
                if parameter_max_len < len(par_val):
                    parameter_max_len = len(par_val)
                parameter_number_num += len(re.findall("\d", par_val))
                parameter_str_num += len(re.findall(r"[a-zA-Z]", par_val))
                parameter_spe_num += len(par_val) - len(re.findall("\d", par_val)) - len(
                    re.findall(r"[a-zA-Z]", par_val))

            try:
                parameter_number_rt = parameter_number_num / par_val_sum
                parameter_str_rt = parameter_str_num / par_val_sum
                parameter_spe_rt = parameter_spe_num / par_val_sum
                feature.append([url_len, url_method, parameter_len, parameter_num,
                                parameter_max_len, parameter_number_num, parameter_str_num,
                                parameter_spe_num, parameter_number_rt, parameter_str_rt,
                                parameter_spe_rt])
            except ZeroDivisionError as err:
                print(err)
                print(data[i])
                continue
    return feature

自定义好后的数据用 numpy 形式保存，方便以后直接使用。
向量化的正常数据：vector_normal.txt
向量化的异常数据：vector_anomalous.txt



同样地，我们也需要整合已经向量化好的数据，并进行打标签操作，划分测试集和训练集。



In [51]:
# 整合数据，打标签
import numpy as np

normal = np.loadtxt("vector_normal.txt")
anomalous = np.loadtxt("vector_anomalous.txt")

all_requests = np.concatenate([normal, anomalous])
X = all_requests

y_normal = np.zeros(shape=(normal.shape[0]), dtype='int')
y_anomalous = np.ones(shape=(anomalous.shape[0]), dtype='int')
y = np.concatenate([y_normal, y_anomalous])

# 划分测试集和训练集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)